# Introducing Deepagents Library by LangChain
While a basic Large Language Model (LLM) agent—one that repeatedly calls external tools—is easy to create, these agents often struggle with long and complex tasks because they lack the ability to plan ahead and manage their work over time. They can be considered “shallow” in their execution.

The deepagents library is designed to overcome this limitation by implementing a general architecture inspired by advanced applications like Deep Research and Claude Code.

This architecture gives agents more depth by combining four key features:

* **A Planning Tool:** Allows the agent to strategically break down a complex task into manageable steps before acting.

* **Sub-Agents:** Enables the main agent to delegate specialized parts of the task to smaller, focused agents.

* **Access to a File System:** Provides persistent memory for saving work-in-progress, notes, and final outputs, allowing the agent to continue where it left off.

* **A Detailed Prompt:** Gives the agent clear instructions, context, and constraints for its long-term objectives.

By providing these foundational components, deepagents makes it easier for developers to build powerful, general-purpose agents that can plan, manage state, and execute complex workflows effectively.

In this article, we’ll take a look at a practical example to see how DeepAgents actually work in action.

## Setting up dependencies

In [36]:
!pip install deepagents tavily-python langchain-google-genai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 3.9 MB/s eta 0:00:00


### Environment Variables
In this tutorial, we’ll use the OpenAI API key to power our Deep Agent. However, for reference, we’ll also show how you can use a Gemini model instead.

You’re free to choose any model provider you prefer — OpenAI, Gemini, Anthropic, or others — as DeepAgents works seamlessly with different backends.

In [31]:
import os
from getpass import getpass
os.environ['TAVILY_API_KEY'] = getpass('Enter Tavily API Key: ')
os.environ['OPENAI_API_KEY'] = getpass('Enter OpenAI API Key: ')
os.environ['GOOGLE_API_KEY'] = getpass('Enter Google API Key: ')

## Importing the necessary libraries

In [10]:
import os
from typing import Literal
from tavily import TavilyClient
from deepagents import create_deep_agent


tavily_client = TavilyClient()

## Tools
Just like regular tool-using agents, a Deep Agent can also be equipped with a set of tools to help it perform tasks.

In this example, we’ll give our agent access to a Tavily Search tool, which it can use to gather real-time information from the web.


In [74]:
from typing import Literal
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent


def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs



## Sub-Agents
Subagents are one of the most powerful features of Deep Agents. They allow the main agent to delegate specific parts of a complex task to smaller, specialized agents — each with its own focus, tools, and instructions. This helps keep the main agent’s context clean and organized while still allowing for deep, focused work on individual subtasks.

In our example, we defined two subagents:

* **policy-research-agent** — a specialized researcher that conducts in-depth analysis on AI policies, regulations, and ethical frameworks worldwide. It uses the internet_search tool to gather real-time information and produces a well-structured, professional report.

* **policy-critique-agent** — an editorial agent responsible for reviewing the generated report for accuracy, completeness, and tone. It ensures that the research is balanced, factual, and aligned with regional legal frameworks.

Together, these subagents enable the main Deep Agent to perform research, analysis, and quality review in a structured, modular workflow.

In [75]:
sub_research_prompt = """
You are a specialized AI policy researcher.
Conduct in-depth research on government policies, global regulations, and ethical frameworks related to artificial intelligence.

Your answer should:
- Provide key updates and trends
- Include relevant sources and laws (e.g., EU AI Act, U.S. Executive Orders)
- Compare global approaches when relevant
- Be written in clear, professional language

Only your FINAL message will be passed back to the main agent.
"""

research_sub_agent = {
    "name": "policy-research-agent",
    "description": "Used to research specific AI policy and regulation questions in depth.",
    "system_prompt": sub_research_prompt,
    "tools": [internet_search],
}

In [76]:
sub_critique_prompt = """
You are a policy editor reviewing a report on AI governance.
Check the report at `final_report.md` and the question at `question.txt`.

Focus on:
- Accuracy and completeness of legal information
- Proper citation of policy documents
- Balanced analysis of regional differences
- Clarity and neutrality of tone

Provide constructive feedback, but do NOT modify the report directly.
"""

critique_sub_agent = {
    "name": "policy-critique-agent",
    "description": "Critiques AI policy research reports for completeness, clarity, and accuracy.",
    "system_prompt": sub_critique_prompt,
}

## System Prompt
Deep Agents include a built-in system prompt that serves as their core set of instructions. This prompt is inspired by the system prompt used in Claude Code and is designed to be more general-purpose, providing guidance on how to use built-in tools like planning, file system operations, and subagent coordination.

However, while the default system prompt makes Deep Agents capable out of the box, it’s highly recommended to define a custom system prompt tailored to your specific use case. Prompt design plays a crucial role in shaping the agent’s reasoning, structure, and overall performance.

In our example, we defined a custom prompt called policy_research_instructions, which transforms the agent into an expert AI policy researcher. It clearly outlines a step-by-step workflow — saving the question, using the research subagent for analysis, writing the report, and optionally invoking the critique subagent for review. It also enforces best practices such as Markdown formatting, citation style, and professional tone to ensure the final report meets high-quality policy standards.

In [77]:
policy_research_instructions = """
You are an expert AI policy researcher and analyst.
Your job is to investigate questions related to global AI regulation, ethics, and governance frameworks.

1️⃣ Save the user's question to `question.txt`
2️⃣ Use the `policy-research-agent` to perform in-depth research
3️⃣ Write a detailed report to `final_report.md`
4️⃣ Optionally, ask the `policy-critique-agent` to critique your draft
5️⃣ Revise if necessary, then output the final, comprehensive report

When writing the final report:
- Use Markdown with clear sections (## for each)
- Include citations in [Title](URL) format
- Add a ### Sources section at the end
- Write in professional, neutral tone suitable for policy briefings
"""

## Main Agent

Here we define our main Deep Agent using the create_deep_agent() function. We initialize the model with **OpenAI’s gpt-4o**, but as shown in the commented-out line, you can easily switch to **Google’s Gemini 2.5 Flash** model if you prefer. The agent is configured with the internet_search tool, our custom policy_research_instructions system prompt, and two subagents — one for in-depth research and another for critique.

By default, DeepAgents internally uses **Claude Sonnet 4.5 (2025-09-29)** as its model if none is explicitly specified, but the library allows full flexibility to integrate OpenAI, Gemini, Anthropic, or other [LLMs supported by LangChain](https://python.langchain.com/docs/integrations/chat/).

In [82]:
model = init_chat_model(model="openai:gpt-4o")
# model = init_chat_model(model="google_genai:gemini-2.5-flash")
agent = create_deep_agent(
    model=model,
    tools=[internet_search],
    system_prompt=policy_research_instructions,
    subagents=[research_sub_agent, critique_sub_agent],
)

## Invoking the Agent

In [83]:
query = "What are the latest updates on the EU AI Act and its global impact?"
result = agent.invoke({"messages": [{"role": "user", "content": query}]})

## Understanding the Deep Agent’s Execution Trace
The printed log shows the step-by-step execution of the Deep Agent as it processes the query “What are the latest updates on the EU AI Act and its global impact?”.

Here’s what’s happening under the hood:

* The agent begins by saving the question to a local file (/question.txt) using the write_file tool.

* It then spawns a subagent (policy-research-agent) through the task tool to perform in-depth policy research.

* The subagent conducts its research and returns a detailed report.

* The main agent writes the final output to /final_report.md, completing the workflow.

* Finally, it sends a message confirming that the report has been generated and is ready for review.

This structured breakdown shows how the Deep Agent coordinates multiple tool calls and subagent tasks — effectively demonstrating long-context reasoning, planning, and memory persistence across multiple steps.

In [84]:
for msg in result["messages"]:
    print(msg)


content='What are the latest updates on the EU AI Act and its global impact?' additional_kwargs={} response_metadata={} id='6571301c-d18e-43b1-a2c4-c41af21d70af'
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 4165, 'total_tokens': 4200, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4096}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CSODuUGJPmB17QuGIKruGdtXhOSwv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--0b73d08c-8ef2-4fa1-9380-f63768f01e91-0' tool_calls=[{'name': 'write_file', 'args': {'file_path': '/question.txt', 'content': 'What are the latest updates on the EU AI Act and its global impact?'}, 'id': 'call_eWhbLzyjF27qhbHY6xCFzI6Y', 'ty

## Parsing Deep Agent Messages into Structured Data
This function organizes the raw messages from a Deep Agent run into separate categories for human input, AI responses, and tool interactions, making it easier to analyze and visualize the agent’s workflow.

In [85]:
from typing import Any

def parse_messages(result: dict):
    data = {
        "human": [],
        "ai": [],
        "tools": [],
    }

    for msg in result.get("messages", []):
        if hasattr(msg, "name"):  # ToolMessage (e.g., name='write_file')
            data["tools"].append({
                "tool_name": getattr(msg, "name", None),
                "tool_call_id": getattr(msg, "tool_call_id", None),
                "content": getattr(msg, "content", None)
            })
        elif hasattr(msg, "tool_calls") and msg.tool_calls:
            # AIMessage with tool calls (the reasoning + tool actions)
            calls = []
            for call in msg.tool_calls:
                calls.append({
                    "name": call["name"],
                    "args": call["args"]
                })
            data["ai"].append({
                "content": getattr(msg, "content", None),
                "tool_calls": calls
            })
        else:
            # HumanMessage or simple AIMessage
            msg_type = type(msg).__name__
            entry = {"type": msg_type, "content": getattr(msg, "content", None)}
            if msg_type.lower().startswith("human"):
                data["human"].append(entry)
            else:
                data["ai"].append(entry)
    return data


In [86]:
parsed = parse_messages(result)


This data represents a parsed summary of the interactions from a Deep Agent workflow. It separates the messages into three categories: human inputs, AI responses, and tool actions. In this example, there were no direct human messages. The AI used tools like write_file and a task subagent to research the latest updates on the EU AI Act, save the question to question.txt, generate a detailed report, and save it to final_report.md. The final entry shows the AI confirming that the research is complete and provides a link to access the report. This structure helps track how the agent processed the task and interacted with tools step by step.

In [87]:
parsed

{'human': [],
 'ai': [],
 'tools': [{'tool_name': None,
   'tool_call_id': None,
   'content': 'What are the latest updates on the EU AI Act and its global impact?'},
  {'tool_name': None, 'tool_call_id': None, 'content': ''},
  {'tool_name': 'write_file',
   'tool_call_id': 'call_eWhbLzyjF27qhbHY6xCFzI6Y',
   'content': 'Updated file /question.txt'},
  {'tool_name': None, 'tool_call_id': None, 'content': ''},
  {'tool_name': 'task',
   'tool_call_id': 'call_W7YCnKyLhoxzp0ffeYB6nK2S',
   'content': 'The EU AI Act is part of the European Commission\'s comprehensive AI strategy, focusing on enhancing investment and governance in AI, which aims to significantly accelerate the adoption of AI technologies across various sectors by 2030. Here are some key updates and global implications based on recent discussions and analyses:\n\n1. **Investment and Implementation Strategy**: The European Commission has announced a substantial funding push with an allocation of approximately $1.16 billion. 